In [1]:
import os
import openai
import datetime
import pandas as pd
import json
import PyPDF2 as pdf
from PyPDF2 import PdfReader
import numpy as np
import time

In [2]:
api_key_file = open("OPENAI_API_KEY", "r")
openai.api_key = api_key_file.read()

### Dataframes

In [5]:
orders_df = pd.DataFrame({"orderID":[1,2,3], "productID":["a","b","c"],"time":[get_time(),get_time(),get_time()],"status":["delivered","pending","issue"]}).set_index("orderID")
orders_df

,productID,time,status
orderID,,,
1,a,2023-07-31 22:38:26.368576,delivered
2,b,2023-07-31 22:38:26.368626,pending
3,c,2023-07-31 22:38:26.368633,issue


In [6]:
tickets_df = pd.DataFrame({"userID":[], "ticket_details":[],"time":[]})
tickets_df

,userID,ticket_details,time


### Functions

In [4]:
def get_time():
    ct = datetime.datetime.now()
    return str(ct)

def get_status(orderID):
    return str(dict(orders_df[orders_df.index == orderID].reset_index().loc[0]))
    
    
def file_ticket(userID, ticket_details):
    global tickets_df
    tickets_df = tickets_df.append({"userID":userID, "ticket_details":ticket_details,"time":get_time()},ignore_index=True)
    return "Ticket Submitted"

### GPT Code

In [7]:
premise = "You’re a kind helpful tech support chatbot. For ticket filings: Ensure that the user provides their UserID when asking for details. If they do not provide their ID, do not let them file a ticket. When the ticket is filed successfuly, do not provide advice."
premise = premise
messages=[
    {"role": "system", "content": premise}
      ]
functions=[{
                "name": "get_time",
                "description": "returns string representation of a datetime object. For reference, this is in Singapore, so be able to guess the time in other time zones when a different country is specified",
                "parameters": {
                    "type": "object",
                    "properties": {
                    },
                    "required": [],
                },
            },
    {
                "name": "file_ticket",
                "description": "files a ticket to a dataframe, logging UserID and ticket_details.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "userID": {"type": "integer"},
                        "ticket_details": {"type": "string"},
                    },
                    "required": ["userID","ticket_details"],
                },
            },
    
    {
                "name": "get_status",
                "description": "gets the status in dictionary form of an order of a specific order ID",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "orderID": {"type": "integer"},
                    },
                    "required": ["orderID"],
                },
            }
]



def conversation(response):
    content = response
    messages.append({"role": "user", "content": content})
    
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions, function_call = "auto",
    )
    message = completion["choices"][0]["message"]
    
    #Functions Block
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        print(message["function_call"]["arguments"])
        arguments = json.loads(message["function_call"]["arguments"])
        
        
        #Time Function
        if function_name == "get_time":
            function_response = get_time()
            
        #Ticket Function
        elif function_name == "file_ticket":
            function_response = file_ticket(arguments["userID"],arguments["ticket_details"])
        
        #Order Status
        elif function_name == "get_status":
            function_response = get_status(arguments["orderID"])
            
        #Error Handling
        else:
            raise NotImplementedError()
        
        messages.append({"role":"function","name":function_name,"content":function_response})
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages = messages,
        )
        answer_response = second_response.choices[0]["message"]["content"].strip()
        messages.append({"role": "assistant", "content": answer_response})
    
    
    #Normal Block
    else:
        chat_response = completion.choices[0].message.content
        messages.append({"role": "assistant", "content": chat_response})
    for i in messages:
        if i['role'] == "user":
            print("User: ", i['content'])
            #print("\n")
        elif i['role'] == "assistant":
            print("ChatGPT: ", i['content'])
        print("\n")
    

In [14]:
conversation("WHAT time is it")



{}


User:  Hello wasyp


ChatGPT:  Hello! How can I assist you today?


User:  Yo I wanna find out bout mah order i need the details


ChatGPT:  Sure, I can help you with that. Please provide me with your order ID so that I can retrieve the details for you.


User:  我的orderID是2




ChatGPT:  Thank you for providing your order ID. I have found the details of your order. Here is what I have:

- Order ID: 2
- Product ID: b
- Time of order: 2023-07-31 22:38:26
- Status: Pending

If you have any further questions or need assistance, please let me know!


User:  我要file ticket


ChatGPT:  Sure, I can help you file a ticket. Please provide me with your User ID and the details of the ticket you would like to file.


User:  WHAT IS TFT THE GAME


ChatGPT:  TFT stands for Teamfight Tactics. It is an online multiplayer strategy game developed by Riot Games. In TFT, players build and manage a team of champions to battle against other players. The goal is to create the strongest team and strategica

In [8]:
tickets_df

,userID,ticket_details,time
